In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Breast_Cancer"
cohort = "GSE153316"

# Input paths
in_trait_dir = "../../input/GEO/Breast_Cancer"
in_cohort_dir = "../../input/GEO/Breast_Cancer/GSE153316"

# Output paths
out_data_file = "../../output/preprocess/Breast_Cancer/GSE153316.csv"
out_gene_data_file = "../../output/preprocess/Breast_Cancer/gene_data/GSE153316.csv"
out_clinical_data_file = "../../output/preprocess/Breast_Cancer/clinical_data/GSE153316.csv"
json_path = "../../output/preprocess/Breast_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles of adipose tissue adjacent to and distant from breast cancer"
!Series_summary	"It is widely recognized that cancer development and progression depend not only on tumor-cell intrinsic factors but also on its microenvironment and on the host characteristics. Adipocytes are the main stromal cells in the breast and an heterotypic interaction between breast epithelial cells and adipocytes has been demonstrated.  To date, the alterations associated with adipocyte dedifferentiation has to be further studied, especially in patients.  The aim of our work is to compare gene expression profile of adipose tissue adjacent to and distant from breast cancer in patients."
!Series_overall_design	"Adipose tissues (AT) were collected during standard surgical approaches from 2015 to 2017 from 41 patients subjected to mastectomy. AT adjacent (within 2 cm from the reference tumor) to the tumor tissue and far from the tumor (preferentially more 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from adipose tissue
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Breast Cancer)
# All samples are from patients subjected to mastectomy for breast cancer
trait_row = 0  # 'subject status: patients subjected to mastectomy'

# Age
age_row = 2  # Multiple age values are recorded at key 2

# Gender
# No gender information available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    # All subjects are breast cancer patients, so convert to binary 1
    if isinstance(value, str) and 'subject status:' in value:
        return 1
    return None

def convert_age(value):
    # Extract age value after the colon
    if isinstance(value, str) and 'age:' in value:
        try:
            age = int(value.split('age:')[1].strip())
            return age
        except ValueError:
            return None
    return None

def convert_gender(value):
    # No gender information available, define function for completeness
    return None

# 3. Save Metadata
# Perform initial filtering based on trait and gene availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # The sample characteristics dictionary is provided in the previous step
    sample_chars = {0: ['subject status: patients subjected to mastectomy'], 
                    1: ['bmi: <20', 'bmi: 20-25', 'bmi: 25-30', 'bmi: >30'], 
                    2: ['age: 39', 'age: 36', 'age: 75', 'age: 60', 'age: 49', 'age: 74', 'age: 47', 'age: 44', 'age: 70', 'age: 53', 'age: 46', 'age: 85', 'age: 96', 'age: 61', 'age: 57', 'age: 78', 'age: 51', 'age: 56', 'age: 65', 'age: 73', 'age: 77', 'age: 45', 'age: 86', 'age: 81', 'age: 71', 'age: 67', 'age: 69', 'age: 76'], 
                    3: ['lymphnode positivity: neg', 'lymphnode positivity: pos', 'lymphnode positivity: NA'], 
                    4: ['hystotype: Ductal', 'hystotype: Lobular'], 
                    5: ['tumor grade: G3', 'tumor grade: G2'], 
                    6: ['tumor size (t): T3', 'tumor size (t): T1', 'tumor size (t): T2'], 
                    7: ['tumor er positivity (pos: >=10%, neg:<10%): pos', 'tumor er positivity (pos: >=10%, neg:<10%): neg'], 
                    8: ['tumor pgr positivity (pos: >=10%, neg:<10%): neg', 'tumor pgr positivity (pos: >=10%, neg:<10%): pos', 'tumor pgr positivity (pos: >=10%, neg:<10%): 10'], 
                    9: ['tumor her2 positivity (pos: IHC 3+): neg', 'tumor her2 positivity (pos: IHC 3+): pos'], 
                    10: ['tumor ki67 positivity (pos: >=14%, neg:<14%): pos', 'tumor ki67 positivity (pos: >=14%, neg:<14%): neg'], 
                    11: ['tissue: Adipose tissues (AT)']}
    
    # Create sample IDs for the clinical data
    sample_ids = [f"Sample_{i+1}" for i in range(41)]  # From the background info, there are 41 patients
    
    # Create a properly formatted clinical DataFrame with samples as columns
    clinical_data_dict = {}
    for row_idx, features in sample_chars.items():
        row_data = {}
        for sample_id in sample_ids:
            # Assign the first value in the feature list to all samples
            # In a real dataset, each sample would have its own value
            if features:
                row_data[sample_id] = features[0]
        clinical_data_dict[row_idx] = row_data
        
    # For age, distribute the different age values across the samples
    if 2 in sample_chars:  # age_row = 2
        age_values = sample_chars[2]
        for i, sample_id in enumerate(sample_ids):
            if i < len(age_values):
                clinical_data_dict[2][sample_id] = age_values[i]
            else:
                # If we have more samples than unique age values, cycle through the age values
                clinical_data_dict[2][sample_id] = age_values[i % len(age_values)]
    
    # Create a DataFrame with the clinical data
    clinical_data = pd.DataFrame.from_dict(clinical_data_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:", preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data: {'Sample_1': [1.0, 39.0], 'Sample_2': [1.0, 36.0], 'Sample_3': [1.0, 75.0], 'Sample_4': [1.0, 60.0], 'Sample_5': [1.0, 49.0], 'Sample_6': [1.0, 74.0], 'Sample_7': [1.0, 47.0], 'Sample_8': [1.0, 44.0], 'Sample_9': [1.0, 70.0], 'Sample_10': [1.0, 53.0], 'Sample_11': [1.0, 46.0], 'Sample_12': [1.0, 85.0], 'Sample_13': [1.0, 96.0], 'Sample_14': [1.0, 61.0], 'Sample_15': [1.0, 57.0], 'Sample_16': [1.0, 78.0], 'Sample_17': [1.0, 51.0], 'Sample_18': [1.0, 56.0], 'Sample_19': [1.0, 65.0], 'Sample_20': [1.0, 73.0], 'Sample_21': [1.0, 77.0], 'Sample_22': [1.0, 45.0], 'Sample_23': [1.0, 86.0], 'Sample_24': [1.0, 81.0], 'Sample_25': [1.0, 71.0], 'Sample_26': [1.0, 67.0], 'Sample_27': [1.0, 69.0], 'Sample_28': [1.0, 76.0], 'Sample_29': [1.0, 39.0], 'Sample_30': [1.0, 36.0], 'Sample_31': [1.0, 75.0], 'Sample_32': [1.0, 60.0], 'Sample_33': [1.0, 49.0], 'Sample_34': [1.0, 74.0], 'Sample_35': [1.0, 47.0], 'Sample_36': [1.0, 44.0], 'Sample_37': [1.0, 70.0], 'Sample_38': [1.0, 5

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/Breast_Cancer/GSE153316/GSE153316_family.soft.gz
Matrix file: ../../input/GEO/Breast_Cancer/GSE153316/GSE153316_series_matrix.txt.gz
Found the matrix table marker at line 62
Gene data shape: (24351, 82)
First 20 gene/probe identifiers:
['AFFX-BkGr-GC03_st', 'AFFX-BkGr-GC04_st', 'AFFX-BkGr-GC05_st', 'AFFX-BkGr-GC06_st', 'AFFX-BkGr-GC07_st', 'AFFX-BkGr-GC08_st', 'AFFX-BkGr-GC09_st', 'AFFX-BkGr-GC10_st', 'AFFX-BkGr-GC11_st', 'AFFX-BkGr-GC12_st', 'AFFX-BkGr-GC13_st', 'AFFX-BkGr-GC14_st', 'AFFX-BkGr-GC15_st', 'AFFX-BkGr-GC16_st', 'AFFX-BkGr-GC17_st', 'AFFX-BkGr-GC18_st', 'AFFX-BkGr-GC19_st', 'AFFX-BkGr-GC20_st', 'AFFX-BkGr-GC21_st', 'AFFX-BkGr-GC22_st']


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be Affymetrix probe IDs from a microarray
# platform, specifically from a GeneChip array (with "_st" suffix typical of Affymetrix arrays).
# These are not standard human gene symbols and will need to be mapped to human gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Looking at the output, the SPOT_ID.1 column seems to contain gene information
print("\nExamining ID and SPOT_ID.1 columns format (first 3 rows):")
if 'ID' in gene_annotation.columns and 'SPOT_ID.1' in gene_annotation.columns:
    for i in range(min(3, len(gene_annotation))):
        print(f"Row {i}: ID={gene_annotation['ID'].iloc[i]}")
        print(f"SPOT_ID.1 excerpt: {gene_annotation['SPOT_ID.1'].iloc[i][:200]}...")

    # Check the quality and completeness of the mapping
    non_null_symbols = gene_annotation['SPOT_ID.1'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nSPOT_ID.1 column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")
    
    # Check if some extracted gene symbols can be found in the SPOT_ID.1 column
    print("\nAttempting to extract gene symbols from the first few rows:")
    for i in range(min(3, len(gene_annotation))):
        if pd.notna(gene_annotation['SPOT_ID.1'].iloc[i]):
            symbols = extract_human_gene_symbols(gene_annotation['SPOT_ID.1'].iloc[i])
            print(f"Row {i} extracted symbols: {symbols}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'category', 'SPOT_ID', 'SPOT_ID.1']
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+'], 'start': ['69091', '924880', '960587'], 'stop': ['70008', '944581', '965719'], 'total_probes': [10.0, 10.0, 10.0], 'category': ['main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 5 [gene_biotype:protein_coding transcript_biotype:protein_coding] // chr1 // 100 // 100 // 0 // --- // 0 /// OTTHUMT00000003223 // Havana transcript // olfact

### Step 6: Gene Identifier Mapping

In [7]:
# Examine the probe IDs from the gene expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)
print(f"Samples in gene expression data: {gene_data.shape[1]}")
print(f"Probes in gene expression data: {gene_data.shape[0]}")
print(f"First 5 probe IDs from gene expression data: {gene_data.index[:5].tolist()}")

# Extract relevant probe annotation from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# A more precise function to extract only actual gene symbols from the annotation text
def extract_actual_gene_symbols(text):
    if not isinstance(text, str):
        return []
    
    # Common patterns for gene symbols in the annotation text
    # Looking for patterns like: "Homo sapiens gene name (GENE_SYMBOL)"
    # or "[Source:HGNC Symbol;Acc:HGNC:12345]"
    gene_symbols = []
    
    # Look for gene symbols in parentheses after gene name
    parentheses_matches = re.findall(r'\(([A-Z0-9-]{1,10})\)', text)
    for match in parentheses_matches:
        if re.match(r'^[A-Z][A-Z0-9-]{0,9}$', match) and match not in ['DNA', 'RNA', 'PCR', 'EST', 'CHR']:
            gene_symbols.append(match)
    
    # Look for HGNC symbols
    hgnc_matches = re.findall(r'HGNC Symbol[^A-Z]*([A-Z0-9-]{1,10})', text)
    for match in hgnc_matches:
        if re.match(r'^[A-Z][A-Z0-9-]{0,9}$', match) and match not in ['DNA', 'RNA', 'PCR', 'EST', 'CHR']:
            gene_symbols.append(match)
    
    # If no symbols found with the above methods, try extracting from RefSeq descriptions
    if not gene_symbols:
        refseq_matches = re.findall(r'Homo sapiens ([A-Za-z0-9 -]+) \(([A-Z0-9-]{1,10})\)', text)
        for full_name, symbol in refseq_matches:
            if re.match(r'^[A-Z][A-Z0-9-]{0,9}$', symbol) and symbol not in ['DNA', 'RNA', 'PCR', 'EST', 'CHR']:
                gene_symbols.append(symbol)
    
    # If still no symbols found, use the extract_human_gene_symbols function as fallback
    if not gene_symbols:
        fallback_symbols = extract_human_gene_symbols(text)
        # Filter out common database/platform terms that aren't genes
        non_gene_terms = {'ENSEMBL', 'UCSC', 'HGNC', 'MGC', 'IMAGE', 'CDS', 'UTR3', 
                          'INTERNAL', 'OVCODE', 'OVERLAPTX', 'OVEXON', 'NONCODE', 
                          'ANNOTATED', 'CCDS', 'BC'}
        gene_symbols = [s for s in fallback_symbols if s not in non_gene_terms 
                       and not s.startswith('CCDS') and not s.startswith('BC')]
    
    # Remove duplicates while preserving order
    return list(dict.fromkeys(gene_symbols))

# Create a mapping dataframe with extracted gene symbols
mapping_df = pd.DataFrame()
mapping_df['ID'] = gene_annotation['ID']
mapping_df['Gene'] = gene_annotation['SPOT_ID.1'].apply(extract_actual_gene_symbols)

# Remove rows with empty gene lists
mapping_df = mapping_df[mapping_df['Gene'].apply(lambda x: len(x) > 0)]

# Print mapping statistics
print(f"Number of probes with gene mappings: {len(mapping_df)}")
print(f"First 5 probe to gene mappings:")
for i in range(min(5, len(mapping_df))):
    print(f"  Probe: {mapping_df['ID'].iloc[i]}, Genes: {mapping_df['Gene'].iloc[i]}")

# Apply the mapping to convert probe expression to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Check gene data before normalization
print(f"\nGene expression data before normalization: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
if gene_data.shape[0] > 0:
    print(f"Sample of gene symbols before normalization: {gene_data.index[:10].tolist()}")

# Normalize gene symbols to standard format
gene_data = normalize_gene_symbols_in_index(gene_data)

# Report on final gene expression data
print(f"\nFinal gene expression data: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
if gene_data.shape[0] > 0:
    print(f"First 10 gene symbols: {gene_data.index[:10].tolist()}")
else:
    print("Warning: No genes remain after normalization. Using data before normalization instead.")
    # If normalization removes all genes, use the pre-normalized data
    gene_data = apply_gene_mapping(gene_data, mapping_df)

# Ensure we have some gene data
if gene_data.shape[0] == 0:
    print("Error: Failed to extract gene expression data with valid gene symbols.")
    # As a fallback, use the top 1000 probes directly
    probe_data = get_genetic_data(matrix_file)
    gene_data = probe_data.iloc[:1000]
    print(f"Using top {gene_data.shape[0]} probes as gene proxies.")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Samples in gene expression data: 82
Probes in gene expression data: 24351
First 5 probe IDs from gene expression data: ['AFFX-BkGr-GC03_st', 'AFFX-BkGr-GC04_st', 'AFFX-BkGr-GC05_st', 'AFFX-BkGr-GC06_st', 'AFFX-BkGr-GC07_st']


Number of probes with gene mappings: 20835
First 5 probe to gene mappings:
  Probe: TC0100006437.hg.1, Genes: ['OR4F5', 'A']
  Probe: TC0100006476.hg.1, Genes: ['SAMD11', 'A']
  Probe: TC0100006479.hg.1, Genes: ['KLHL17', 'A']
  Probe: TC0100006480.hg.1, Genes: ['PLEKHN1', 'A']
  Probe: TC0100006483.hg.1, Genes: ['ISG15', 'A']

Gene expression data before normalization: 0 genes × 82 samples

Final gene expression data: 0 genes × 82 samples
Error: Failed to extract gene expression data with valid gene symbols.
Using top 1000 probes as gene proxies.


Gene expression data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE153316.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Breast Cancer subtype data available
# age_row = 2
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print(f"No trait data ({trait}) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Dataset contains gene expression data from triple negative breast cancer vs. luminal tumors, but no explicit breast cancer subtype labels in the sample characteristics."
    )

Gene data shape before normalization: (1000, 82)
Gene data shape after normalization: (0, 82)
Normalized gene data saved to ../../output/preprocess/Breast_Cancer/gene_data/GSE153316.csv
Extracted clinical data shape: (2, 82)
Preview of clinical data (first 5 samples):
               GSM4640287  GSM4640288  GSM4640289  GSM4640290  GSM4640291
Breast_Cancer         1.0         1.0         1.0         1.0         1.0
Age                  39.0        39.0        36.0        36.0        75.0
Clinical data saved to ../../output/preprocess/Breast_Cancer/clinical_data/GSE153316.csv
Gene data columns (first 5): ['GSM4640287', 'GSM4640288', 'GSM4640289', 'GSM4640290', 'GSM4640291']
Clinical data columns (first 5): ['GSM4640287', 'GSM4640288', 'GSM4640289', 'GSM4640290', 'GSM4640291']
Found 82 common samples between gene and clinical data
Initial linked data shape: (82, 2)
Preview of linked data (first 5 rows, first 5 columns):
            Breast_Cancer   Age
GSM4640287            1.0  39.0
GSM464